# 2.1 Suggest dates

## 1. Is it a useful feature for our landlords?

First of all, let's see how many request entries are available in the database.

```sql
SELECT count(*)
FROM   booking
R: 10000 entries
```

In order to evaluate how useful is the new feature, we will analyse how many times "new_move_in_at" was used.

```sql
SELECT count(new_move_in_at)
FROM   booking
WHERE  new_move_in_at IS NOT NULL;
R: 325 entries
```

We can say that out of 10000 total entries, "new_move_in_at" was used 325 times (3.25% of entries).

> **Feedback**:
We tell you the feature was released recently. Did you try to understand when? Otherwise, you may *underestimate* usage.

Going into detail, we will analyse out of those 325 entries, how many resulted in a positive outcome (booking).
From my understading, if current_state equal to 'accepted', 'paid', 'confirmed', a booking was made.

> **Feedback**:
Refer back to the exercise. The landlord can accept or reject (or expire) and then the student can pay or not. There are two steps.

```sql
SELECT COUNT(*)
FROM   booking
WHERE  new_move_in_at IS NOT NULL 
       AND current_state IN('accepted', 'paid', 'confirmed');
R: 52 entries
```

From 325 entries of new_move_in_at, 52 resulted in a booking (16%). This is when students accepted, paid or confirmed the new move in/move out dates set by the landlord.

> **Feedback**:
Refer back to the exercise. The Landlord rejects, accepts or suggests new dates instead. The student pays.

To conclude, this feature isn't very used by landlords (325 out of 10000 cases) but is useful in terms of date flexibility set by landlords (52 positive cases from 325).

> **Feedback**:
You need to understand first when the feature was deployed, understand the data to go more in depth.

## 2. Is there any pattern in the suggested dates? 

### Could this be used to improve the feature, or create new ones?

Let's check how many distinct (unique) dates we can find from landlord suggestions.

```sql
/* Move in dates */
SELECT COUNT(DISTINCT new_move_in_at) AS dates
FROM   booking
R: 82
```

There are 82 distinct dates out of 325 entries.

> **Feedback**:
Why only specific dates? Why not specific days of the month, of even specific months?

> Does it change depending on the date of request?

```sql
/* Move out dates */
SELECT COUNT(DISTINCT new_move_out_at) AS dates
FROM   booking
R: 127
```
There are 127 distinct dates out of 325 entries.

From these results, we can go into more detail analysing which dates were most suggested.

```sql
/* Move in dates */
SELECT new_move_in_at, COUNT(new_move_in_at) AS date_occurrence
FROM   booking
GROUP  BY new_move_in_at
ORDER  BY date_occurrence DESC
LIMIT  10
R:
'2017-09-01 00:00:00	101'
'2017-10-01 00:00:00	29'
'2017-08-01 00:00:00	20'
'2017-08-31 00:00:00	16'
'2017-09-16 00:00:00	8'
'2017-09-15 00:00:00	7'
'2017-08-16 00:00:00	7'
'2017-09-03 00:00:00	6'
'2017-08-29 00:00:00	5'
'2017-09-05 00:00:00	5'
```

```sql
/* Move out dates */
SELECT new_move_out_at, COUNT(new_move_out_at) AS date_occurrence
FROM   booking
GROUP  BY new_move_out_at
ORDER  BY date_occurrence DESC
LIMIT  10
R:
'2018-01-31 00:00:00	82'
'2018-06-30 00:00:00	23'
'2018-08-31 00:00:00	16'
'2018-07-31 00:00:00	12'
'2018-02-28 00:00:00	11'
'2017-06-30 00:00:00	9'
'2017-12-31 00:00:00	8'
'2017-12-30 00:00:00	7'
'2017-11-30 00:00:00	6'
'2018-01-17 00:00:00	6'
```

Analysing the results, we can see that September 1st is clearly a pattern (101 cases) as a new move in date.

> **Feedback**:
The pattern is there, you just need to dig a bit further. It's not about the specific date. 

> *Hint: Think about how landlords manage occupancy.*

Being this date so common, let's find which dates are usually suggested as move out date.

```sql
SELECT new_move_out_at, COUNT(new_move_out_at) AS date_occurences
FROM   booking
WHERE  new_move_in_at = '20170901'
GROUP  BY new_move_out_at
ORDER  BY date_occurences DESC
LIMIT  10;
R:
'2018-01-31 00:00:00	36'
'2018-06-30 00:00:00	10'
'2018-08-31 00:00:00	8'
'2017-12-30 00:00:00	6'
'2017-12-23 00:00:00	4'
'2018-07-31 00:00:00	4'
'2017-09-30 00:00:00	3'
'2018-02-28 00:00:00	3'
'2018-01-30 00:00:00	2'
'2018-05-31 00:00:00	2'
```

Analyzing the dates when the new move in date is September 1st (101 cases), we can conclude that in 36 cases the move out date is at January 31st and in 10 cases its June 30th, which is equivalent to the end of the first and second school semester, accordingly.

The subsequent most common dates are close to the top 2 dates mentioned above. 

The data confirms a seasonal pattern in move in dates at the beginning of the 1st semester (new_move_in_at : 2017-09-01) and at the end of 1st semester (new_move_out_at : 2018-01-31).
Considering these patterns, it could be suggested predefined date intervals according to 1st semester length and/or full school year (new_move_out_at : 2018-06-30). 

> **Feedback:**
This is legit and true, but does not tell the whole story. Why would landlords care about the school calendar? 

> Is this what happens when *you* are looking for a place? Se hint above.

## 3. Does it influence the acceptance rate of our landlords?

First of all, is important to say that according to my interpretation and understading of all features and consequent data, 
I decided to use acceptance_at to answer this question, instead of using current_state and its states (e.g. 'accepted').

> **Feedback**:
This is a solid choice, good reasoning. You are expected to understand that `current_state` is not a final and use the timestamps.

This being said, in order to analyse the new feature influence in booking's acceptance rate, let's use the accepted_at.

```sql
SELECT COUNT(accepted_at)
FROM   booking
R: 2901
```

2901 accepted bookings out of 10000 total entries (~29%).

> **Feedback**:
Partially correct. You were expected to understand the date of release of the new feature to compare acceptance rates.

```sql
From the suggested cases:
SELECT COUNT(accepted_at)
FROM booking
WHERE new_move_in_at IS NOT NULL
R: 321
```

321 accepted bookings out of 325 new_move_in_at not null cases (~99%).

> **Feedback**:
Again, partially correct. Refer back to the exercise: the landlords either accept the booking or suggest new dates.

> In fact, suggesting new dates is *a type of acceptance* (those 4 remaining cases are weird).

> The number doesn't seem too high, shouldn't we investigate further such result?

Solely by acceptance_at, we can say that the new feature have a strong positive influence, increasing from 29% to 99%.

> **Feedback**:
You were expected to understand the date of release of the new feature and compare acceptance rates.

## 4. Does it influence the payment rate of our students?

Is important to say that according to my interpretation and understading of all features and consequent data, 
I decided to use paid_at to answer this question, instead of using current_state and its states (e.g. 'paid').
This being said, in order to analyse the new feature influence in booking's payment rate, let's use paid_at.

> **Feedback**:
Again, strong reasoning. I'm affraid you are missing a point, though.

```sql
SELECT COUNT(*)
FROM booking
WHERE paid_at IS NOT NULL;
R: 1258
```
1258 paid bookings out of 10000 total entries (~13%).

> **Feedback**:
You're missing the point. Refer back to the exercise: the student can only pay if the landlord accepted the booking.

```sql
SELECT COUNT(*)
FROM booking
WHERE paid_at IS NOT NULL AND new_move_in_at IS NOT NULL;
R: 51
```

51 paid bookings out of 325 new_move_in_at not null cases (~16%).

> **Feedback**:
Here, since you use the not null condition, you are considering only accepted bookings, thus using a different base.

Using this feature, we can see a positive influence in the payment rate of bookings by students.


> **Feedback**:
This exercise is not correct.

## 5. What more data would you require to evaluate the feature and why?

Approaching this problem without prior knowledge, could be useful to have a detailed definition of what each feature means and how each feature interacts with one another, 
as there were some doubts related to what each state meant in current_state and how exactly they were related with other features.

> **Feedback**:
You're absolutely right, good point. We can do some changes in the future but this is mostly on purpose.

> For us, it's very important to understand *how the candidates think* and they state their hypotheses and test them.

For example in question 3, to calculate the acceptance rate of our landlords could we assume
the states "paid" and "confirmed" to be included as part of the percentage accepted? 

> **Feedback**:
You should and you did - refer to the exercise (the way it works section) and/or look at the data. 

> You have bookings with accepted_at not null with current state paid, payment-expired, etc. *Exploratory data analysis is key.*

Would be interesting to extend, over time, the use of this feature to gather more data, since its a relatively recent (less than a year).

> **Feedback**:
Good point.

> But again, the feature is even more recent than that. Again, *exploratory data analysis is key*.

Some suggestions could be made regarding booking time intervals (as seen in question 2) and then see the "acceptance rate" of those predefined suggestions.

> **Feedback**:
Refer back to the exercise.

> "The landlord receives the booking request and can accept or reject. As part of the acceptance, the landlord can suggest new dates."


# 2.2 Price recommendation

## 1. What are your main conclusions regarding the results of the feature?

Analysing each figure, one can say that:

* There's an increase of booking requests since the beginning of the test;
* With price adjustments, this feature created interest in published offers that got no booking requests until then (overpriced offers);
* Despite the increase in booking requests, the majority of offers (more than 500) still have no booking requests;
* Offers that suffered a price decrease in the interval of 0 to 100 got more booking requests, although also being the majority of total offers. In cases were the decrease was of 100, the demand even surpassed the supply;
* The fact that offers with higher price decreases didn't had many requests might be related to landlord's high evaluation of its offer. Even with a greater price adjustment it is still not suitable for most people looking for an appartment/room/bed;
* Seeing the price decrease percent figure its clear that price adjustments are made accordingly to each offer's price. A price decrease of 20% is predominant although it may not have the greatest success ratio (total requests / total offers);
* After July 2017 there was an significant decrease on the average click position, which might mean that clients found a more interesting offer much faster than before. Top search offers became more relevant over time (lower average click position);

> **Feedback**:
I don't understand this one.

* Similarly, we can conclude that top search results became/are more relevant (more total clicks).

> **Feedback**:
Can you clarify?

* Since the implementation of this feature (August 2017), offers with price recommendation became more interesting to students and had more total clicks (massive increase). In the other hand, this increase could also be correlated with searches prior to the beginning of a new semester.

> **Feedback**:
I don't understand how you are reasoning about search and clicks. 

> The idea is that these offers were ranking higher on search prior to the price recommendation.

> **Feedback**:
Was price recommendation good or bad? How would you integrate all data points to tell a story?

2. What other data would you need to evaluate the feature?

It would be interesting to analyse the acceptance rates by landlords regarding booking requests with new prices. Especially in cases of booking rejection, gather information regarding price reduction flexibility. In general, although knowing that this feature created a positive impact in "obsolete" offers, see how much of this new interest resulted in success cases (booked offers).

> **Feedback**:
On point, this is a very good answer, with a very good understanding of the underlying problem.

3. Can you design an experiment leading to more conclusive results? If so, please describe it.

We can separate results by offer type, analyse price considering offer's characteristics and location (e.g. proximity to university, near public transports, average and variance price values of the area in which the offer is located), gathering average and variance of prices for each offer type.

> **Feedback**:
How would this help evaluate the current algorithm?

Making a similar analysis, we can create a classification method (e.g. Decision Tree), training our data to return a price evaluation interval (e.g. 0 - 'cheap', 1 - 'normal', 2 - 'expensive', set according to average and variance price values) considering all relevant features and historic data.

> **Feedback**:
Wouldn't this be creating a new algorithm, instead of evaluating the one we currently have? 

> In the case we would do this, how would you evualutate one algorithm against the other?

> Also, I don't think the methodology here is correct. Can you try to describe *exactly* what you would do, step-by-step? It will help you.

Thus, not booked offers, for a certain time threshold (defined by Data Analytics team), with similar characteristics could have a new price suggestion (adequate price per offer) taking into account similar booked offers, therefore reducing price discrepancies between "real price" and landlord set overprice. 
Then we could analyse the interest created in those offers by amount of clicks and booking rates, considering rates of "normal" offers and "adapted" offers.

> **Feedback**:
This approach is more promising, albeit lacks substance. The billion dollar question being, what are similar offers?